In [2]:
# ============================================================
# 01 — FILTRE METEO (IPYNB)
# -> copie les images OK dans out/image_met_ok/
# ============================================================

import os, re, csv, shutil
from datetime import datetime
import numpy as np

# --- chemins (IPYNB) ---
ROOT = os.getcwd()
IMAGES_DIR = os.path.join(ROOT, "data", "images")
METEO_CSV  = os.path.join(ROOT, "data", "meteo.csv")

OUT_DIR    = os.path.join(ROOT, "out")
OUT_OK     = os.path.join(OUT_DIR, "image_met_ok")
OUT_CSV    = os.path.join(OUT_DIR, "meteo_selection.csv")

os.makedirs(OUT_OK, exist_ok=True)

# --- paramètres ---
MAX_DT_S = 15 * 60  # ±15 min

MAX_WIND   = 10000
MAX_RAIN   = 10000
MIN_IRRAD  = 0
TMIN, TMAX = 0, 10000

# --- helpers ---
def img_dt(fname):
    m = re.search(r"(\d{8}T\d{4})TU", fname)
    if not m: return None
    return datetime.strptime(m.group(1), "%Y%m%dT%H%M")

def ffloat(x):
    try: return float(x)
    except: return np.nan

def met_ok(m):
    if np.isnan(m["Vvent"]) or m["Vvent"] > MAX_WIND: return False, "vent"
    if np.isnan(m["Rain"])  or m["Rain"]  > MAX_RAIN: return False, "pluie"
    if np.isnan(m["Irrad"]) or m["Irrad"] < MIN_IRRAD: return False, "irradiance"
    if np.isnan(m["Tair"])  or not (TMIN <= m["Tair"] <= TMAX): return False, "temp"
    return True, "OK"

# --- charge meteo ---
met_t, met = [], []
with open(METEO_CSV, "r", newline="") as f:
    r = csv.DictReader(f)
    for row in r:
        dt = datetime.strptime(row["timestamp"], "%Y%m%dT%H%M")
        met_t.append(dt)
        met.append({
            "dt": dt,
            "Tair":  ffloat(row["T_air"]),
            "Irrad": ffloat(row["Irrad"]),
            "Vvent": ffloat(row["V_vent"]),
            "Rain":  ffloat(row["Pluie"]),
        })
met_t = np.array(met_t)

def nearest(dt_img):
    dts = np.array([abs((t - dt_img).total_seconds()) for t in met_t])
    i = int(np.argmin(dts))
    return met[i], float(dts[i])

# --- run ---
imgs = sorted([f for f in os.listdir(IMAGES_DIR) if f.lower().endswith((".jpg",".jpeg",".png"))])

rows = []
kept = 0
for fname in imgs:
    dt = img_dt(fname)
    if dt is None:
        rows.append({"image": fname, "decision": "skip_name", "reason": "no_TU_timestamp"})
        continue

    m, dt_s = nearest(dt)
    time_ok = dt_s <= MAX_DT_S
    ok, why = met_ok(m)

    if time_ok and ok:
        shutil.copy2(os.path.join(IMAGES_DIR, fname), os.path.join(OUT_OK, fname))
        kept += 1
        decision, reason = "keep", "OK"
    else:
        decision = "reject"
        reason = f"dt={int(dt_s)}s" if not time_ok else why

    rows.append({
        "image": fname,
        "img_utc": dt.isoformat(),
        "meteo_utc": m["dt"].isoformat(),
        "dt_s": int(dt_s),
        "Tair": m["Tair"],
        "Irrad": m["Irrad"],
        "Vvent": m["Vvent"],
        "Rain": m["Rain"],
        "decision": decision,
        "reason": reason,
    })

with open(OUT_CSV, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=rows[0].keys())
    w.writeheader(); w.writerows(rows)

print("kept =", kept, "| out =", OUT_OK, "| csv =", OUT_CSV)


kept = 48 | out = /Users/alexandremichaux/Documents/UCA/Cours/Stage/projet/Stage-M1MV/FUSE/out/image_met_ok | csv = /Users/alexandremichaux/Documents/UCA/Cours/Stage/projet/Stage-M1MV/FUSE/out/meteo_selection.csv


In [ ]:
# run_fuse_physical.py
# ------------------------------------------------------------
# Lance fuse_detect sur un dossier d’images et sauvegarde
# une figure diagnostic par image
# ------------------------------------------------------------

import os
from fuse_physical import fuse_detect

# ------------------------------------------------------------
# PATHS
# ------------------------------------------------------------
IMG_DIR = "out/image_met_ok"
OUT_FIG = "out/figures"

os.makedirs(OUT_FIG, exist_ok=True)


ROI = (300, 125, 450, 225)
S_MIN = 2

# ------------------------------------------------------------
# LOOP
# ------------------------------------------------------------
images = sorted(
    f for f in os.listdir(IMG_DIR)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
)

for fname in images:
    img_path = os.path.join(IMG_DIR, fname)
    fig_path = os.path.join(
        OUT_FIG,
        os.path.splitext(fname)[0] + "_diag.png"
    )

    detected, mask, metrics = fuse_detect(
        image_path=img_path,
        ROI=ROI,
        S_min=S_MIN,
        show=False,
        save_fig=fig_path,
    )

    print(f"{fname} | detected={detected} | {metrics}")


20231220T0009TU_GPVHOU2_br.jpg | detected=True | {'area_px': 728, 'mean_score': 1.356415867805481, 'reason': 'ok'}
20231220T0039TU_GPVHOU2_br.jpg | detected=True | {'area_px': 781, 'mean_score': 1.399941086769104, 'reason': 'ok'}
20231220T0109TU_GPVHOU2_br.jpg | detected=True | {'area_px': 690, 'mean_score': 1.332101583480835, 'reason': 'ok'}
20231220T0139TU_GPVHOU2_br.jpg | detected=True | {'area_px': 540, 'mean_score': 1.395115613937378, 'reason': 'ok'}
20231220T0209TU_GPVHOU2_br.jpg | detected=True | {'area_px': 454, 'mean_score': 1.4906823635101318, 'reason': 'ok'}
20231220T0239TU_GPVHOU2_br.jpg | detected=True | {'area_px': 226, 'mean_score': 1.3729984760284424, 'reason': 'ok'}
20231220T0309TU_GPVHOU2_br.jpg | detected=True | {'area_px': 427, 'mean_score': 1.349167823791504, 'reason': 'ok'}
20231220T0339TU_GPVHOU2_br.jpg | detected=True | {'area_px': 284, 'mean_score': 3.0455546379089355, 'reason': 'ok'}
20231220T0409TU_GPVHOU2_br.jpg | detected=True | {'area_px': 379, 'mean_score